In [1]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import requests
from pandas.io.json import json_normalize
import json
import urllib.request

from difflib import SequenceMatcher

In [4]:
station_loc = pd.read_csv('cleaned_data/station_loc.csv')

In [5]:
station_loc.head(2)

,name,lat,lon,region_id,region
0,Eads St & 15th St S,38.858971,-77.053230,41,"Arlington, VA"
1,Crystal Dr & 20th St S,38.856425,-77.049232,41,"Arlington, VA"


In [63]:
df_2011 = pd.read_csv('cleaned_data/df_2011.csv', parse_dates = ['start_date', 'end_date'])
df_2012 = pd.read_csv('cleaned_data/df_2012.csv', parse_dates = ['start_date', 'end_date'])
df_2013 = pd.read_csv('cleaned_data/df_2013.csv', parse_dates = ['start_date', 'end_date'])
df_2014 = pd.read_csv('cleaned_data/df_2014.csv', parse_dates = ['start_date', 'end_date'])
df_2015 = pd.read_csv('cleaned_data/df_2015.csv', parse_dates = ['start_date', 'end_date'])
df_2016 = pd.read_csv('cleaned_data/df_2016.csv', parse_dates = ['start_date', 'end_date'])
df_2017 = pd.read_csv('cleaned_data/df_2017.csv', parse_dates = ['start_date', 'end_date'])

<h3> For the years 2012 and 2013, create a separate dataframe with station info </h5>

In [20]:
start_stations_2012 = df_2012['start_station'].unique()
end_stations_2012 = df_2012['end_station'].unique()

start_stations_2013 = df_2013['start_station'].unique()
end_stations_2013 = df_2013['end_station'].unique()

stations_2012 = np.union1d(start_stations_2012, end_stations_2012)
stations_2013 = np.union1d(start_stations_2013, end_stations_2013)

In [21]:
len(stations_2012), len(stations_2013)

(191, 305)

In [22]:
stations_2012 = pd.DataFrame(stations_2012).rename(columns = {0: 'station'})
stations_2013 = pd.DataFrame(stations_2013).rename(columns = {0: 'station'})

In [24]:
stations_2012.head()
stations_2013.head()

,station
0,10th & E St NW
1,10th & Florida Ave NW
2,10th & K St NW
3,10th & Monroe St NE
4,10th & U St NW


In [25]:
# Add "region", "lat", and "lon" columns to stations_2011 using stations_loc

stations_2012['region'] = np.zeros(stations_2012.shape[0])
stations_2012['lat'] = np.zeros(stations_2012.shape[0])
stations_2012['lon'] = np.zeros(stations_2012.shape[0])

for i in stations_2012.index:
    for j in station_loc.index:
        if stations_2012.loc[i,'station'] == station_loc.loc[j, 'name']:
            stations_2012.loc[i, 'lat'] = station_loc.loc[j, 'lat']
            stations_2012.loc[i, 'lon'] = station_loc.loc[j, 'lon']
            stations_2012.loc[i, 'region'] = station_loc.loc[j, 'region']

In [46]:
stations_2013['region'] = np.zeros(stations_2013.shape[0])
stations_2013['lat'] = np.zeros(stations_2013.shape[0])
stations_2013['lon'] = np.zeros(stations_2013.shape[0])

for i in stations_2013.index:
    for j in station_loc.index:
        if stations_2013.loc[i,'station'] == station_loc.loc[j, 'name']:
            stations_2013.loc[i, 'lat'] = station_loc.loc[j, 'lat']
            stations_2013.loc[i, 'lon'] = station_loc.loc[j, 'lon']
            stations_2013.loc[i, 'region'] = station_loc.loc[j, 'region']

In [50]:
stations_2012.head()

,station,region,lat,lon
0,10th & E St NW,"Washington, DC",38.895914,-77.026064
1,10th & Monroe St NE,"Washington, DC",38.932457,-76.993534
2,10th & U St NW,"Washington, DC",38.917200,-77.025900
3,10th St & Constitution Ave NW,"Washington, DC",38.893028,-77.026013
4,11th & F St NW,"Washington, DC",38.897857,-77.026975


In [51]:
stations_2013.head()

,station,region,lat,lon
0,10th & E St NW,"Washington, DC",38.895914,-77.026064
1,10th & Florida Ave NW,"Washington, DC",38.920387,-77.025672
2,10th & K St NW,"Washington, DC",38.902400,-77.026220
3,10th & Monroe St NE,"Washington, DC",38.932457,-76.993534
4,10th & U St NW,"Washington, DC",38.917200,-77.025900


In [36]:
num_missing_2012 = stations_2012[stations_2012['lat']==0].count()
num_missing_2013 = stations_2012[stations_2012['lat']==0].count()

In [38]:
num_missing_2012, num_missing_2013, len(stations_2012), len(stations_2013)

(station    39
 region     39
 lat        39
 lon        39
 dtype: int64,
 station    39
 region     39
 lat        39
 lon        39
 dtype: int64,
 191,
 305)

In [47]:
# Create list of missing stations 

missing_stations_2012 = []
for i in stations_2012.index:
    if stations_2012.loc[i,'lat'] == 0 or stations_2012.loc[i,'lon'] == 0:
        missing_stations_2012.append(stations_2012.loc[i,'station'])

missing_stations_2013 = []
for i in stations_2013.index:
    if stations_2013.loc[i,'lat'] == 0 or stations_2013.loc[i,'lon'] == 0:
        missing_stations_2013.append(stations_2013.loc[i,'station'])

In [41]:
missing_stations_2011 = ['Crystal City Metro / 18th & Bell St',
 '21st & M St NW',
 'Eastern Market Metro / Pennsylvania Ave & 7th St SE',
 'Connecticut Ave & Newark St NW / Cleveland Park',
 '18th & Eads St.',
 '19th & L St NW',
 '23rd & Crystal Dr',
 'Aurora Hills Community Ctr/18th & Hayes St',
 'S Joyce & Army Navy Dr',
 'Georgia Ave and Fairmont St NW',
 '20th & Crystal Dr',
 'S Glebe & Potomac Ave',
 'USDA / 12th & Independence Ave SW',
 '27th & Crystal Dr',
 'Pentagon City Metro / 12th & S Hayes St',
 '12th & Army Navy Dr',
 '26th & S Clark St',
 '15th & Crystal Dr',
 'Eads & 22nd St S',
 '1st & N St  SE',
 'Lynn & 19th St North',
 'N Rhodes & 16th St N',
 'Rosslyn Metro / Wilson Blvd & Ft Myer Dr',
 'Wilson Blvd & Franklin Rd',
 '11th & H St NE']

In [49]:
len (np.intersect1d(missing_stations_2012, missing_stations_2011)), len (np.intersect1d(missing_stations_2013, missing_stations_2011))

(25, 25)

Nice. 25 of the stations missing from 2012 and 2013 are missing from 2011. <br>
But we already caught a lot of the missing stations from 2011.

<h3> Now, for all years, create a separate dataframe with station info </h3>

We will automate the process carried out above

In [64]:
dataframes = [df_2012, df_2013, df_2014, df_2015, df_2016, df_2017]


# for each year, array with all stations
stations = []

for df in dataframes:
    start = df['start_station'].unique()
    end = df['end_station'].unique()
    both = np.union1d(start,end)
    stations.append(both)

In [65]:
for item in stations:
    print (len(item))

191
305
346
357
435
487


In [68]:
stations_dataframes = []

for item in stations: 
    stations_dataframes.append(pd.DataFrame(item).rename(columns = {0: 'station'}))

In [69]:
stations_dataframes[0].head()

,station
0,10th & E St NW
1,10th & Monroe St NE
2,10th & U St NW
3,10th St & Constitution Ave NW
4,11th & F St NW


In [70]:
for item in stations_dataframes:
    item['region'] = np.zeros(item.shape[0])
    item['lat'] = np.zeros(item.shape[0])
    item['lon'] = np.zeros(item.shape[0])

    for i in item.index:
        for j in station_loc.index: 
            if item.loc[i,'station'] == station_loc.loc[j, 'name']:
                item.loc[i, 'lat'] = station_loc.loc[j, 'lat']
                item.loc[i, 'lon'] = station_loc.loc[j, 'lon']
                item.loc[i, 'region'] = station_loc.loc[j, 'region']


In [71]:
len(stations_dataframes)

6

In [72]:
stations_2012 = stations_dataframes[0]
stations_2013 = stations_dataframes[1]
stations_2014 = stations_dataframes[2]
stations_2015 = stations_dataframes[3]
stations_2016 = stations_dataframes[4]
stations_2017 = stations_dataframes[5]

In [75]:
len(stations_2012), len(stations_2013), len(stations_2014), len(stations_2015), len(stations_2016), len(stations_2017) 

(191, 305, 346, 357, 435, 487)

In [82]:
# Count total number of stations + number of stations with missing coordinates per year
for df in stations_dataframes:
    print (len(df), df[df['lat']==0].count()['station'])


191 39
305 65
346 82
357 84
435 97
487 103


In [83]:
stations_2012.to_csv('cleaned_data/stations_2012.csv', index = False)
stations_2013.to_csv('cleaned_data/stations_2013.csv', index = False)
stations_2014.to_csv('cleaned_data/stations_2014.csv', index = False)
stations_2015.to_csv('cleaned_data/stations_2015.csv', index = False)
stations_2016.to_csv('cleaned_data/stations_2016.csv', index = False)
stations_2017.to_csv('cleaned_data/stations_2017.csv', index = False)
